In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model

In [110]:
train_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_c40/train'
validation_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_c40/val'

In [139]:
test_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_c40/test'

In [5]:
import os
total = 0
for root, dirs, files in os.walk('C:/Users/Administrator/Desktop/Final Project/data_croped_raw/test/original_sequences/actors'):
   total += len(files)

In [6]:
total

1884

In [7]:
train_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                  
                                  )
test_datagen  = ImageDataGenerator( rescale = 1.0/255,
                                  
                                  )

In [8]:
testing_datagen = ImageDataGenerator( rescale=1.0/255)

In [81]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='binary',
                                                    target_size = (224, 224))

Found 4792 images belonging to 72 classes.


In [111]:
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=128,
                                                         class_mode='binary',              
                                                         target_size = (224, 224))

Found 1032 images belonging to 14 classes.


In [140]:
test_generator =  testing_datagen.flow_from_directory(test_dir,
                                                         batch_size=128,
                                                          class_mode='binary',              
                                                         shuffle=False,
                                                         target_size = (224, 224))

Found 7834 images belonging to 2 classes.


In [12]:
from tensorflow.keras.applications import InceptionV3
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed5')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [13]:
len(pre_trained_model.layers)

311

In [14]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784
__________________________________________________________________________________________________


In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers,models
from tensorflow.keras import Model
def create_model():
    
 
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  x = layers.Dropout(0.4)(x) 

  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(128, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.4)(x) 
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(64, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x) 

  # Add a final sigmoid layer for classification
  x = layers.Dense  (1, activation='sigmoid')(x) 
  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = Adam(), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

  return model


In [16]:
def create_model2():
  # Flatten the output layer to 1 dimension

  flatten_layer = layers.Flatten()
  drop_layer_1 = layers.Dropout(0.4)

  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # Add a dropout rate of 0.2
  drop_layer_2 = layers.Dropout(0.2)
  dense_layer_2 = layers.Dense(512, activation='relu')
    
  drop_layer_3= layers.Dropout(0.2)

  dense_layer_3 = layers.Dense(128, activation='relu')
    
  drop_layer_4= layers.Dropout(0.2)
  
  # Add a final sigmoid layer for classification
  prediction_layer = layers.Dense  (1, activation='sigmoid')

  model = models.Sequential([
      pre_trained_model,
      flatten_layer,
#      drop_layer_1,
#      dense_layer_1,
#      drop_layer_2,
#      dense_layer_2,
#      drop_layer_3,
#      dense_layer_3,
#      drop_layer_4,
      prediction_layer
  ])

  model.compile(optimizer = Adam(), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

  return model

In [17]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [18]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [25]:
model=create_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [26]:
import os
checkpoint_path = 'C:/Users/Administrator/Desktop/Final Project/mostafa/training_1'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 verbose=1)

In [27]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',restore_best_weights = True, patience=5)

In [28]:
#tf.debugging.set_log_device_placement(True)
with tf.device('GPU:0'):
  history = model.fit(
              train_generator,
              validation_data = validation_generator,
              #edit 
              steps_per_epoch = 305,
              epochs = 10,
              #edit
              validation_steps = 56,
              callbacks=[cp_callback,callback],
              verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 305 steps, validate for 56 steps
Epoch 1/10
304/305 [============================>.] - ETA: 0s - loss: 0.5544 - accuracy: 0.7886
Epoch 00001: val_accuracy improved from -inf to 0.80622, saving model to C:/Users/Administrator/Desktop/Final Project/mostafa/training_1
INFO:tensorflow:Assets written to: C:/Users/Administrator/Desktop/Final Project/mostafa/training_1\assets
305/305 [==============================] - 185s 608ms/step - loss: 0.5539 - accuracy: 0.7886 - val_loss: 0.5870 - val_accuracy: 0.8062
Epoch 2/10
304/305 [============================>.] - ETA: 0s - loss: 0.3687 - accuracy: 0.8247
Epoch 00002: val_accuracy improved from 0.80622 to 0.82185, saving model to C:/Users/Administrator/Desktop/Final Project/mostafa/training_1
INFO:tensorflow:Assets written to: C:/Users/Administrator/Desktop/Final Project/mostafa/training_1\assets
305/305 [==============================] - 179s 588ms/step - loss: 0.3688 - accuracy: 0.824

In [34]:
from tensorflow.keras.models import load_model

In [35]:
new=load_model('model83_c40.h5')

In [39]:
new.evaluate(train_generator)

  ...
    to  
  ['...']
306/306 [==============================] - 115s 377ms/step - loss: 0.4371 - accuracy: 0.8385


[0.4370751733483832, 0.8385106]

In [43]:
import numpy as np

In [136]:
Y_pred = new.predict_generator(test_generator)
y_pred = np.array([1 * (x[0]>=0.5) for x in Y_pred])

In [137]:
print(len(y_pred[y_pred==0])/len(y_pred)*100)

90.81967213114754


In [71]:
np.unique(test_generator.classes)

array([0, 1])

In [ ]:
#Confution Matrix and Classification Report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
Y_pred = new.predict_generator(test_generator)
y_pred = [1 * (x[0]>=0.5) for x in Y_pred]

cm = confusion_matrix(test_generator.classes, y_pred)
print('Confusion Matrix')
print(cm)
target_names=['fake','real']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


In [73]:
#model.save('best_model_raw.h5')

In [79]:
#model.save('model_raw_93.h5')